# Προχωρημένα Θέματα Βάσεων Δεδομένων

**Ονοματεπώνυμο:** Κωνσταντίνος Διβριώτης

**ΑΜ:** 03114140

## Query 3: 

Χρησιμοποιώντας ως αναφορά τα δεδομένα της απογραφής 2010 για τον πληθυσμό και εκείνα της απογραφής του 2015 για το εισόδημα ανα νοικοκυριό, να υπολογίσετε για κάθε περιοχή του Los Angeles τα παρακάτω:
- Το μέσο ετήσιο εισόδημα ανά άτομο
- Την αναλογία συνολικού αριθμού εγκλημάτων ανά άτομο

In [1]:
from pyspark.sql import SparkSession
from sedona.spark import *

spark = SparkSession \
    .builder \
    .appName("CensusDataAnalysis") \
    .getOrCreate()

sedona = SedonaContext.create(spark)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
118,application_1738075734771_0119,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
DATA_BUCKET = "s3://initial-notebook-data-bucket-dblab-905418150721"
GROUP_BUCKET = "s3://groups-bucket-dblab-905418150721/group15"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Διάβασμα και Επισκόπηση αρχείων εισόδου

In [3]:
from pyspark.sql.types import StructField, StructType, StringType
from pyspark.sql.functions import regexp_replace, col

income_schema = StructType([
    StructField("Zip Code", StringType()),
    StructField("Community", StringType()),
    StructField("Estimated Median Income", StringType())
])

income_data = spark.read.csv(f"{DATA_BUCKET}/LA_income_2015.csv", header=True, schema=income_schema)

# Μετατροπή του Estimated Median Income σε αριθμητική μορφή
income_data = income_data \
    .withColumn(
        "Estimated Median Income",
        regexp_replace(col("Estimated Median Income"), "[$,]", "").cast("float")
    ) \
    .select("Zip Code", "Estimated Median Income")

income_data.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------------------+
|Zip Code|Estimated Median Income|
+--------+-----------------------+
|   90001|                33887.0|
|   90002|                30413.0|
|   90003|                30805.0|
|   90004|                40612.0|
|   90005|                31142.0|
+--------+-----------------------+
only showing top 5 rows

In [4]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DoubleType

# Ορισμός του schema των dataset
crimes_schema = StructType([
    StructField("DR_NO", StringType()),
    StructField("Date Rptd", StringType()),
    StructField("DATE OCC", StringType()),
    StructField("TIME OCC", StringType()),
    StructField("AREA", IntegerType()),
    StructField("AREA NAME", StringType()),
    StructField("Rpt Dist No", StringType()),
    StructField("Part 1-2", IntegerType()),
    StructField("Crm Cd", IntegerType()),
    StructField("Crm Cd Desc", StringType()),
    StructField("Mocodes", StringType()),
    StructField("Vict Age", IntegerType()),
    StructField("Vict Sex", StringType()),
    StructField("Vict Descent", StringType()),
    StructField("Premis Cd", StringType()),
    StructField("Premis Desc", StringType()),
    StructField("Weapon Used Cd", IntegerType()),
    StructField("Weapon Desc", StringType()),
    StructField("Status", StringType()),
    StructField("Status Desc", StringType()),
    StructField("Crm Cd 1", IntegerType()),
    StructField("Crm Cd 2", IntegerType()),
    StructField("Crm Cd 3", IntegerType()),
    StructField("Crm Cd 4", IntegerType()),
    StructField("LOCATION", StringType()),
    StructField("Cross Street", StringType()),
    StructField("LAT", DoubleType()),
    StructField("LON", DoubleType())
])

# Διαβάζουμε τα 2 datasets (2010-2019 και 2020-σήμερα) και τα συνενώνουμε σε 1
crime_data_2010_2019 = spark.read.csv(f"{DATA_BUCKET}/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, schema=crimes_schema)
crime_data_2020_present = spark.read.csv(f"{DATA_BUCKET}/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, schema=crimes_schema)
crime_data = crime_data_2010_2019.union(crime_data_2020_present)

# Μετατρέπουμε τις στήλες LAT, LON σε geometry με το ST_POINT
crime_data = crime_data \
                .withColumn("geom", ST_Point("LON", "LAT")) \
                .filter(col("geom") != ST_Point(0, 0)) \
                .select("DR_NO", "geom")

crime_data.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+
|    DR_NO|                geom|
+---------+--------------------+
|001307355|POINT (-118.2695 ...|
|011401303|POINT (-118.3962 ...|
|070309629|POINT (-118.2524 ...|
|090631215|POINT (-118.3295 ...|
|100100501|POINT (-118.2488 ...|
+---------+--------------------+
only showing top 5 rows

In [5]:
from pyspark.sql.functions import sum, col

blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true") \
            .load(f"{DATA_BUCKET}/2010_Census_Blocks.geojson") \
            .selectExpr("explode(features) as features") \
            .select("features.*")

# Preprocessing των δεδομένων του Census:
# Για κάθε περιοχή του στο Census, υπάρχουν πολλαπλές
# εγγραφές για COMM, ZCTA10. Συνεπώς, πριν κάνουμε το
# join με το Income, πρέπει να κάνουμε group by για να
# υπολογίσουμε το συνολικό πληθυσμό, το πλήθος των
# νοικοκυριών και την καλυπτόμενη περιοχή ανά COMM, ZIP Code
blocks_data = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                    blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type") \
            .filter(
                col("COMM").isNotNull() & (col("CITY") == "Los Angeles") &
                (col("HOUSING10") > 0) & (col("POP_2010") > 0)
            ) \
            .groupBy("COMM", "ZCTA10") \
            .agg(
                sum("POP_2010").alias("Population"),
                sum("HOUSING10").alias("Households"),
                ST_Union_Aggr("geometry").alias("geometry")
            ) \
            .select("COMM", "ZCTA10", "Population", "Households", "geometry")

blocks_data.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------+----------+----------+--------------------+
|         COMM|ZCTA10|Population|Households|            geometry|
+-------------+------+----------+----------+--------------------+
|  West Vernon| 90037|     30161|      8556|POLYGON ((-118.29...|
|        Palms| 90232|      1533|       917|MULTIPOLYGON (((-...|
|   Silverlake| 90039|     10521|      4753|MULTIPOLYGON (((-...|
|     Westwood| 90025|      2443|      1317|MULTIPOLYGON (((-...|
|South Carthay| 90035|      9428|      5250|POLYGON ((-118.36...|
+-------------+------+----------+----------+--------------------+
only showing top 5 rows

In [6]:
blocks_data_description_schema = StructType([
    StructField("field", StringType()),
    StructField("type", StringType()),
    StructField("meaning", StringType())
])

blocks_data_description = spark.read.csv(f"{DATA_BUCKET}/2010_Census_Blocks_fields.csv", header=True, schema=blocks_data_description_schema)

blocks_data_description \
        .filter(col("field").isin("COMM", "ZCTA10", "HOUSING10", "POP_2010", "geometry")) \
        .show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------+-------------------------------------------------------------------------+
|field    |type    |meaning                                                                  |
+---------+--------+-------------------------------------------------------------------------+
|COMM     |string  |Unincorporated area community name and LA City neighborhood              |
|HOUSING10|long    |2010 housing (PL 94-171 Redistricting Data Summary File - Total Housing) |
|POP_2010 |long    |Population (PL 94-171 Redistricting Data Summary File - Total Population)|
|ZCTA10   |string  |Zip Code Tabulation Area                                                 |
|geometry |geometry|Geometry of the block                                                    |
+---------+--------+-------------------------------------------------------------------------+

## Υλοποίηση με DataFrame

In [34]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Kάνουμε join με το Income με βάση το ZIP Code,
# οπότε έχουμε το μέσο ετήσιο εισόδημα ανά νοικοκυριό
# για κάθε COMM, ZIP Code.
# Κάνουμε group by με το COMM για να βρούμε το συνολικό
# εισόδημα και τον συνολικό πληθυσμό ανά COMM.
result = blocks_data \
            .join(income_data, col("ZCTA10") == col("Zip Code")) \
            .groupBy("COMM") \
            .agg(
                sum("Population").alias("Population"),
                sum(col("Estimated Median Income") * col("Households")).alias("Total Income"),
                ST_Union_Aggr("geometry").alias("geometry")
            ) \
            .select("COMM", "Population", "Total Income", "geometry")

# Τέλος, κάνουμε join με τα εγκλήματα με βάση το geometry, δηλαδή
# το POINT του εγκλήματος βρίσκεται εντός του POLYGON της περιοχής.
# Κάνουμε group by με το COMM (και τα Population, Total Income τα οποία είναι
# ίδια ανά εγγραφή) και υπολογίζουμε το συνολικό πλήθος των εγκλημάτων.
# Μετά διαιρούμε με τον πληθυσμό για να βρούμε το εισόδημα ανά άτομο
# και το πλήθος εγκλημάτων ανά άτομο.
result = crime_data \
            .join(
                result,
                ST_Within(col("geom"), col("geometry"))
            ) \
            .groupBy("COMM", "Population", "Total Income") \
            .agg(
                count("*").alias("Total Crimes")
            ) \
            .withColumn("Income per Person", col("Total Income") / col("Population")) \
            .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
            .orderBy("COMM") \
            .select("COMM", "Income per Person", "Crimes per Person")

result.show(truncate=False)

end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+------------------+-------------------+
|COMM                   |Income per Person |Crimes per Person  |
+-----------------------+------------------+-------------------+
|Adams-Normandie        |8791.458301453711 |0.7148686559551135 |
|Alsace                 |11239.50136425648 |0.5416098226466576 |
|Angeles National Forest|33079.58823529412 |0.4117647058823529 |
|Angelino Heights       |18427.059814658805|0.5732940185341197 |
|Arleta                 |12110.779474388612|0.4264509064363061 |
|Atwater Village        |28481.236967160792|0.5288318320448259 |
|Baldwin Hills          |17303.906408241663|0.9950061114021302 |
|Bel Air                |63041.33942621959 |0.39922527539038855|
|Beverly Crest          |60947.48978754819 |0.3689607087195472 |
|Beverlywood            |29267.82118405155 |0.5084977849375755 |
|Boyle Heights          |8494.108286861341 |0.6171887393378809 |
|Brentwood              |60846.854461055365|0.4058638814936173 |
|Brookside              |

In [35]:
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 31.09 seconds

In [36]:
result.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (33)
+- Sort (32)
   +- Exchange (31)
      +- HashAggregate (30)
         +- Exchange (29)
            +- HashAggregate (28)
               +- Project (27)
                  +- RangeJoin (26)
                     :- Union (7)
                     :  :- Project (3)
                     :  :  +- Filter (2)
                     :  :     +- Scan csv  (1)
                     :  +- Project (6)
                     :     +- Filter (5)
                     :        +- Scan csv  (4)
                     +- Filter (25)
                        +- ObjectHashAggregate (24)
                           +- Exchange (23)
                              +- ObjectHashAggregate (22)
                                 +- Project (21)
                                    +- BroadcastHashJoin Inner BuildRight (20)
                                       :- ObjectHashAggregate (15)
                                       :  +- Exchange (14)
                                     

Θα δοκιμάσουμε να αναγκάσουμε το Spark να χρησιμοποιήσει διαφορετικές στρατηγικές, ώστε να συγκρίνουμε την απόδοσή τους.

### 1. BROADCAST

In [7]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Kάνουμε join με το Income με βάση το ZIP Code,
# οπότε έχουμε το μέσο ετήσιο εισόδημα ανά νοικοκυριό
# για κάθε COMM, ZIP Code.
# Κάνουμε group by με το COMM για να βρούμε το συνολικό
# εισόδημα και τον συνολικό πληθυσμό ανά COMM.
result = blocks_data \
            .join(income_data.hint("BROADCAST"), col("ZCTA10") == col("Zip Code")) \
            .groupBy("COMM") \
            .agg(
                sum("Population").alias("Population"),
                sum(col("Estimated Median Income") * col("Households")).alias("Total Income"),
                ST_Union_Aggr("geometry").alias("geometry")
            ) \
            .select("COMM", "Population", "Total Income", "geometry")

# Τέλος, κάνουμε join με τα εγκλήματα με βάση το geometry, δηλαδή
# το POINT του εγκλήματος βρίσκεται εντός του POLYGON της περιοχής.
# Κάνουμε group by με το COMM (και τα Population, Total Income τα οποία είναι
# ίδια ανά εγγραφή) και υπολογίζουμε το συνολικό πλήθος των εγκλημάτων.
# Μετά διαιρούμε με τον πληθυσμό για να βρούμε το εισόδημα ανά άτομο
# και το πλήθος εγκλημάτων ανά άτομο.
result = crime_data \
            .join(
                result,
                ST_Within(col("geom"), col("geometry"))
            ) \
            .groupBy("COMM", "Population", "Total Income") \
            .agg(
                count("*").alias("Total Crimes")
            ) \
            .withColumn("Income per Person", col("Total Income") / col("Population")) \
            .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
            .select("COMM", "Income per Person", "Crimes per Person")

result.show()

end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------------------+-------------------+
|               COMM| Income per Person|  Crimes per Person|
+-------------------+------------------+-------------------+
|       Hancock Park|21538.792826380406| 0.7797133123352832|
|        Playa Vista| 50264.47143177235| 0.5004481290611696|
|    Hollywood Hills| 43023.70025098602|  0.747615632843313|
|       Toluca Woods|24517.584657534248| 0.6301369863013698|
|         West Adams|10768.299623210249| 0.7500753579502637|
|       Century City|45617.760134566866|  0.632968881412952|
|     Harbor Gateway|16152.767540362767|0.46035977675901935|
|          Thai Town|26851.472085561498| 0.5305882352941177|
|Palisades Highlands| 66867.44038612054| 0.1878424210800939|
|            Sunland| 26184.01260332687| 0.4484131033778957|
|    Harvard Heights| 11820.77328771164|  0.771694885257507|
|    Atwater Village|28481.236967160792| 0.5288318320448259|
|   West Los Angeles| 39729.36421669107|   0.61900439238653|
|            Pacoima|   

In [8]:
elapsed_time = end_time - start_time
print(f"Time taken for strategy \"BROADCAST\": {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken for strategy "BROADCAST": 33.36 seconds

In [9]:
result.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (31)
+- HashAggregate (30)
   +- Exchange (29)
      +- HashAggregate (28)
         +- Project (27)
            +- RangeJoin (26)
               :- Union (7)
               :  :- Project (3)
               :  :  +- Filter (2)
               :  :     +- Scan csv  (1)
               :  +- Project (6)
               :     +- Filter (5)
               :        +- Scan csv  (4)
               +- Filter (25)
                  +- ObjectHashAggregate (24)
                     +- Exchange (23)
                        +- ObjectHashAggregate (22)
                           +- Project (21)
                              +- BroadcastHashJoin Inner BuildRight (20)
                                 :- ObjectHashAggregate (15)
                                 :  +- Exchange (14)
                                 :     +- ObjectHashAggregate (13)
                                 :        +- Project (12)
                                 :           +- Filter (11)
     

### 2. MERGE

In [7]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Kάνουμε join με το Income με βάση το ZIP Code,
# οπότε έχουμε το μέσο ετήσιο εισόδημα ανά νοικοκυριό
# για κάθε COMM, ZIP Code.
# Κάνουμε group by με το COMM για να βρούμε το συνολικό
# εισόδημα και τον συνολικό πληθυσμό ανά COMM.
result = blocks_data.hint("MERGE") \
            .join(income_data.hint("MERGE"), col("ZCTA10") == col("Zip Code")) \
            .groupBy("COMM") \
            .agg(
                sum("Population").alias("Population"),
                sum(col("Estimated Median Income") * col("Households")).alias("Total Income"),
                ST_Union_Aggr("geometry").alias("geometry")
            ) \
            .select("COMM", "Population", "Total Income", "geometry")

# Τέλος, κάνουμε join με τα εγκλήματα με βάση το geometry, δηλαδή
# το POINT του εγκλήματος βρίσκεται εντός του POLYGON της περιοχής.
# Κάνουμε group by με το COMM (και τα Population, Total Income τα οποία είναι
# ίδια ανά εγγραφή) και υπολογίζουμε το συνολικό πλήθος των εγκλημάτων.
# Μετά διαιρούμε με τον πληθυσμό για να βρούμε το εισόδημα ανά άτομο
# και το πλήθος εγκλημάτων ανά άτομο.
result = result \
            .join(
                crime_data,
                ST_Within(col("geom"), col("geometry"))
            ) \
            .groupBy("COMM", "Population", "Total Income") \
            .agg(
                count("*").alias("Total Crimes")
            ) \
            .withColumn("Income per Person", col("Total Income") / col("Population")) \
            .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
            .select("COMM", "Income per Person", "Crimes per Person")

result.show()

end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------------------+-------------------+
|               COMM| Income per Person|  Crimes per Person|
+-------------------+------------------+-------------------+
|       Hancock Park|21538.792826380406| 0.7797133123352832|
|        Playa Vista| 50264.47143177235| 0.5004481290611696|
|    Hollywood Hills| 43023.70025098602|  0.747615632843313|
|       Toluca Woods|24517.584657534248| 0.6301369863013698|
|         West Adams|10768.299623210249| 0.7500753579502637|
|       Century City|45617.760134566866|  0.632968881412952|
|     Harbor Gateway|16152.767540362767|0.46035977675901935|
|          Thai Town|26851.472085561498| 0.5305882352941177|
|Palisades Highlands| 66867.44038612054| 0.1878424210800939|
|            Sunland| 26184.01260332687| 0.4484131033778957|
|    Harvard Heights| 11820.77328771164|  0.771694885257507|
|    Atwater Village|28481.236967160792| 0.5288318320448259|
|   West Los Angeles| 39729.36421669107|   0.61900439238653|
|            Pacoima|   

In [8]:
elapsed_time = end_time - start_time
print(f"Time taken for strategy \"MERGE\": {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken for strategy "MERGE": 32.95 seconds

In [9]:
result.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (34)
+- HashAggregate (33)
   +- Exchange (32)
      +- HashAggregate (31)
         +- Project (30)
            +- RangeJoin (29)
               :- Filter (21)
               :  +- ObjectHashAggregate (20)
               :     +- Exchange (19)
               :        +- ObjectHashAggregate (18)
               :           +- Project (17)
               :              +- SortMergeJoin Inner (16)
               :                 :- Sort (10)
               :                 :  +- Exchange (9)
               :                 :     +- ObjectHashAggregate (8)
               :                 :        +- Exchange (7)
               :                 :           +- ObjectHashAggregate (6)
               :                 :              +- Project (5)
               :                 :                 +- Filter (4)
               :                 :                    +- Generate (3)
               :                 :                       +- Filter (2)
  

### 3. SHUFFLE HASH

In [7]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Kάνουμε join με το Income με βάση το ZIP Code,
# οπότε έχουμε το μέσο ετήσιο εισόδημα ανά νοικοκυριό
# για κάθε COMM, ZIP Code.
# Κάνουμε group by με το COMM για να βρούμε το συνολικό
# εισόδημα και τον συνολικό πληθυσμό ανά COMM.
result = blocks_data \
            .join(income_data.hint("SHUFFLE_HASH"), col("ZCTA10") == col("Zip Code")) \
            .groupBy("COMM") \
            .agg(
                sum("Population").alias("Population"),
                sum(col("Estimated Median Income") * col("Households")).alias("Total Income"),
                ST_Union_Aggr("geometry").alias("geometry")
            ) \
            .select("COMM", "Population", "Total Income", "geometry")

# Τέλος, κάνουμε join με τα εγκλήματα με βάση το geometry, δηλαδή
# το POINT του εγκλήματος βρίσκεται εντός του POLYGON της περιοχής.
# Κάνουμε group by με το COMM (και τα Population, Total Income τα οποία είναι
# ίδια ανά εγγραφή) και υπολογίζουμε το συνολικό πλήθος των εγκλημάτων.
# Μετά διαιρούμε με τον πληθυσμό για να βρούμε το εισόδημα ανά άτομο
# και το πλήθος εγκλημάτων ανά άτομο.
result = crime_data \
            .join(
                result,
                ST_Within(col("geom"), col("geometry"))
            ) \
            .groupBy("COMM", "Population", "Total Income") \
            .agg(
                count("*").alias("Total Crimes")
            ) \
            .withColumn("Income per Person", col("Total Income") / col("Population")) \
            .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
            .select("COMM", "Income per Person", "Crimes per Person")

result.show()

end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------------------+-------------------+
|               COMM| Income per Person|  Crimes per Person|
+-------------------+------------------+-------------------+
|       Hancock Park|21538.792826380406| 0.7797133123352832|
|        Playa Vista| 50264.47143177235| 0.5004481290611696|
|    Hollywood Hills| 43023.70025098602|  0.747615632843313|
|       Toluca Woods|24517.584657534248| 0.6301369863013698|
|         West Adams|10768.299623210249| 0.7500753579502637|
|       Century City|45617.760134566866|  0.632968881412952|
|     Harbor Gateway|16152.767540362767|0.46035977675901935|
|          Thai Town|26851.472085561498| 0.5305882352941177|
|Palisades Highlands| 66867.44038612054| 0.1878424210800939|
|            Sunland| 26184.01260332687| 0.4484131033778957|
|    Harvard Heights| 11820.77328771164|  0.771694885257507|
|    Atwater Village|28481.236967160792| 0.5288318320448259|
|   West Los Angeles| 39729.36421669107|   0.61900439238653|
|            Pacoima|   

In [8]:
elapsed_time = end_time - start_time
print(f"Time taken for strategy \"SHUFFLE_HASH\": {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken for strategy "SHUFFLE_HASH": 28.10 seconds

In [9]:
result.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (32)
+- HashAggregate (31)
   +- Exchange (30)
      +- HashAggregate (29)
         +- Project (28)
            +- RangeJoin (27)
               :- Union (7)
               :  :- Project (3)
               :  :  +- Filter (2)
               :  :     +- Scan csv  (1)
               :  +- Project (6)
               :     +- Filter (5)
               :        +- Scan csv  (4)
               +- Filter (26)
                  +- ObjectHashAggregate (25)
                     +- Exchange (24)
                        +- ObjectHashAggregate (23)
                           +- Project (22)
                              +- ShuffledHashJoin Inner BuildRight (21)
                                 :- Exchange (16)
                                 :  +- ObjectHashAggregate (15)
                                 :     +- Exchange (14)
                                 :        +- ObjectHashAggregate (13)
                                 :           +- Project (12)
    

### 4. SHUFFLE REPLICATE NL

In [7]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Kάνουμε join με το Income με βάση το ZIP Code,
# οπότε έχουμε το μέσο ετήσιο εισόδημα ανά νοικοκυριό
# για κάθε COMM, ZIP Code.
# Κάνουμε group by με το COMM για να βρούμε το συνολικό
# εισόδημα και τον συνολικό πληθυσμό ανά COMM.
result = income_data.hint("SHUFFLE_REPLICATE_NL") \
            .join(blocks_data, col("ZCTA10") == col("Zip Code")) \
            .groupBy("COMM") \
            .agg(
                sum("Population").alias("Population"),
                sum(col("Estimated Median Income") * col("Households")).alias("Total Income"),
                ST_Union_Aggr("geometry").alias("geometry")
            ) \
            .select("COMM", "Population", "Total Income", "geometry")

# Τέλος, κάνουμε join με τα εγκλήματα με βάση το geometry, δηλαδή
# το POINT του εγκλήματος βρίσκεται εντός του POLYGON της περιοχής.
# Κάνουμε group by με το COMM (και τα Population, Total Income τα οποία είναι
# ίδια ανά εγγραφή) και υπολογίζουμε το συνολικό πλήθος των εγκλημάτων.
# Μετά διαιρούμε με τον πληθυσμό για να βρούμε το εισόδημα ανά άτομο
# και το πλήθος εγκλημάτων ανά άτομο.
result = crime_data \
            .join(
                result,
                ST_Within(col("geom"), col("geometry"))
            ) \
            .groupBy("COMM", "Population", "Total Income") \
            .agg(
                count("*").alias("Total Crimes")
            ) \
            .withColumn("Income per Person", col("Total Income") / col("Population")) \
            .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
            .select("COMM", "Income per Person", "Crimes per Person")

result.show()

end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------------------+-------------------+
|               COMM| Income per Person|  Crimes per Person|
+-------------------+------------------+-------------------+
|       Hancock Park|21538.792826380406| 0.7797133123352832|
|        Playa Vista| 50264.47143177235| 0.5004481290611696|
|    Hollywood Hills| 43023.70025098602|  0.747615632843313|
|       Toluca Woods|24517.584657534248| 0.6301369863013698|
|         West Adams|10768.299623210249| 0.7500753579502637|
|       Century City|45617.760134566866|  0.632968881412952|
|     Harbor Gateway|16152.767540362767|0.46035977675901935|
|          Thai Town|26851.472085561498| 0.5305882352941177|
|Palisades Highlands| 66867.44038612054| 0.1878424210800939|
|            Sunland| 26184.01260332687| 0.4484131033778957|
|    Harvard Heights| 11820.77328771164|  0.771694885257507|
|    Atwater Village|28481.236967160792| 0.5288318320448259|
|   West Los Angeles| 39729.36421669107|   0.61900439238653|
|            Pacoima|   

In [8]:
elapsed_time = end_time - start_time
print(f"Time taken for strategy \"SHUFFLE_REPLICATE_NL\": {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken for strategy "SHUFFLE_REPLICATE_NL": 27.93 seconds

In [9]:
result.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (30)
+- HashAggregate (29)
   +- Exchange (28)
      +- HashAggregate (27)
         +- Project (26)
            +- RangeJoin (25)
               :- Union (7)
               :  :- Project (3)
               :  :  +- Filter (2)
               :  :     +- Scan csv  (1)
               :  +- Project (6)
               :     +- Filter (5)
               :        +- Scan csv  (4)
               +- Filter (24)
                  +- ObjectHashAggregate (23)
                     +- Exchange (22)
                        +- ObjectHashAggregate (21)
                           +- Project (20)
                              +- CartesianProduct Inner (19)
                                 :- Project (10)
                                 :  +- Filter (9)
                                 :     +- Scan csv  (8)
                                 +- ObjectHashAggregate (18)
                                    +- Exchange (17)
                                       +- Objec

### 5. SHUFFLE_HASH + BROADCAST

In [7]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Kάνουμε join με το Income με βάση το ZIP Code,
# οπότε έχουμε το μέσο ετήσιο εισόδημα ανά νοικοκυριό
# για κάθε COMM, ZIP Code.
# Κάνουμε group by με το COMM για να βρούμε το συνολικό
# εισόδημα και τον συνολικό πληθυσμό ανά COMM.
result = blocks_data \
            .join(income_data.hint("SHUFFLE_HASH"), col("ZCTA10") == col("Zip Code")) \
            .groupBy("COMM") \
            .agg(
                sum("Population").alias("Population"),
                sum(col("Estimated Median Income") * col("Households")).alias("Total Income"),
                ST_Union_Aggr("geometry").alias("geometry")
            ) \
            .select("COMM", "Population", "Total Income", "geometry")

# Τέλος, κάνουμε join με τα εγκλήματα με βάση το geometry, δηλαδή
# το POINT του εγκλήματος βρίσκεται εντός του POLYGON της περιοχής.
# Κάνουμε group by με το COMM (και τα Population, Total Income τα οποία είναι
# ίδια ανά εγγραφή) και υπολογίζουμε το συνολικό πλήθος των εγκλημάτων.
# Μετά διαιρούμε με τον πληθυσμό για να βρούμε το εισόδημα ανά άτομο
# και το πλήθος εγκλημάτων ανά άτομο.
result = crime_data \
            .join(
                result.hint("BROADCAST"),
                ST_Within(col("geom"), col("geometry"))
            ) \
            .groupBy("COMM", "Population", "Total Income") \
            .agg(
                count("*").alias("Total Crimes")
            ) \
            .withColumn("Income per Person", col("Total Income") / col("Population")) \
            .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
            .select("COMM", "Income per Person", "Crimes per Person")

result.show()

end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------------------+-------------------+
|               COMM| Income per Person|  Crimes per Person|
+-------------------+------------------+-------------------+
|       Hancock Park|21538.792826380406| 0.7797133123352832|
|        Playa Vista| 50264.47143177235| 0.5004481290611696|
|    Hollywood Hills| 43023.70025098602|  0.747615632843313|
|       Toluca Woods|24517.584657534248| 0.6301369863013698|
|         West Adams|10768.299623210249| 0.7500753579502637|
|       Century City|45617.760134566866|  0.632968881412952|
|     Harbor Gateway|16152.767540362767|0.46035977675901935|
|          Thai Town|26851.472085561498| 0.5305882352941177|
|Palisades Highlands| 66867.44038612054| 0.1878424210800939|
|            Sunland| 26184.01260332687| 0.4484131033778957|
|    Harvard Heights| 11820.77328771164|  0.771694885257507|
|    Atwater Village|28481.236967160792| 0.5288318320448259|
|   West Los Angeles| 39729.36421669107|   0.61900439238653|
|            Pacoima|   

In [8]:
elapsed_time = end_time - start_time
print(f"Time taken for strategy \"SHUFFLE_HASH\": {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken for strategy "SHUFFLE_HASH": 28.80 seconds

In [9]:
result.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (33)
+- HashAggregate (32)
   +- Exchange (31)
      +- HashAggregate (30)
         +- Project (29)
            +- BroadcastIndexJoin (28)
               :- Union (7)
               :  :- Project (3)
               :  :  +- Filter (2)
               :  :     +- Scan csv  (1)
               :  +- Project (6)
               :     +- Filter (5)
               :        +- Scan csv  (4)
               +- SpatialIndex (27)
                  +- Filter (26)
                     +- ObjectHashAggregate (25)
                        +- Exchange (24)
                           +- ObjectHashAggregate (23)
                              +- Project (22)
                                 +- ShuffledHashJoin Inner BuildRight (21)
                                    :- Exchange (16)
                                    :  +- ObjectHashAggregate (15)
                                    :     +- Exchange (14)
                                    :        +- ObjectHashAggre

### 6. SHUFFLE_REPLICATE_NL + BROADCAST

In [7]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Kάνουμε join με το Income με βάση το ZIP Code,
# οπότε έχουμε το μέσο ετήσιο εισόδημα ανά νοικοκυριό
# για κάθε COMM, ZIP Code.
# Κάνουμε group by με το COMM για να βρούμε το συνολικό
# εισόδημα και τον συνολικό πληθυσμό ανά COMM.
result = income_data.hint("SHUFFLE_REPLICATE_NL") \
            .join(blocks_data, col("ZCTA10") == col("Zip Code")) \
            .groupBy("COMM") \
            .agg(
                sum("Population").alias("Population"),
                sum(col("Estimated Median Income") * col("Households")).alias("Total Income"),
                ST_Union_Aggr("geometry").alias("geometry")
            ) \
            .select("COMM", "Population", "Total Income", "geometry")


# Τέλος, κάνουμε join με τα εγκλήματα με βάση το geometry, δηλαδή
# το POINT του εγκλήματος βρίσκεται εντός του POLYGON της περιοχής.
# Κάνουμε group by με το COMM (και τα Population, Total Income τα οποία είναι
# ίδια ανά εγγραφή) και υπολογίζουμε το συνολικό πλήθος των εγκλημάτων.
# Μετά διαιρούμε με τον πληθυσμό για να βρούμε το εισόδημα ανά άτομο
# και το πλήθος εγκλημάτων ανά άτομο.
result = crime_data \
            .join(
                result.hint("BROADCAST"),
                ST_Within(col("geom"), col("geometry"))
            ) \
            .groupBy("COMM", "Population", "Total Income") \
            .agg(
                count("*").alias("Total Crimes")
            ) \
            .withColumn("Income per Person", col("Total Income") / col("Population")) \
            .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
            .select("COMM", "Income per Person", "Crimes per Person")

result.show()

end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------------------+-------------------+
|               COMM| Income per Person|  Crimes per Person|
+-------------------+------------------+-------------------+
|       Hancock Park|21538.792826380406| 0.7797133123352832|
|        Playa Vista| 50264.47143177235| 0.5004481290611696|
|    Hollywood Hills| 43023.70025098602|  0.747615632843313|
|       Toluca Woods|24517.584657534248| 0.6301369863013698|
|         West Adams|10768.299623210249| 0.7500753579502637|
|       Century City|45617.760134566866|  0.632968881412952|
|     Harbor Gateway|16152.767540362767|0.46035977675901935|
|          Thai Town|26851.472085561498| 0.5305882352941177|
|Palisades Highlands| 66867.44038612054| 0.1878424210800939|
|            Sunland| 26184.01260332687| 0.4484131033778957|
|    Harvard Heights| 11820.77328771164|  0.771694885257507|
|    Atwater Village|28481.236967160792| 0.5288318320448259|
|   West Los Angeles| 39729.36421669107|   0.61900439238653|
|            Pacoima|   

In [8]:
elapsed_time = end_time - start_time
print(f"Time taken for strategy \"SHUFFLE_REPLICATE_NL\": {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken for strategy "SHUFFLE_REPLICATE_NL": 30.92 seconds

In [9]:
result.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (31)
+- HashAggregate (30)
   +- Exchange (29)
      +- HashAggregate (28)
         +- Project (27)
            +- BroadcastIndexJoin (26)
               :- Union (7)
               :  :- Project (3)
               :  :  +- Filter (2)
               :  :     +- Scan csv  (1)
               :  +- Project (6)
               :     +- Filter (5)
               :        +- Scan csv  (4)
               +- SpatialIndex (25)
                  +- Filter (24)
                     +- ObjectHashAggregate (23)
                        +- Exchange (22)
                           +- ObjectHashAggregate (21)
                              +- Project (20)
                                 +- CartesianProduct Inner (19)
                                    :- Project (10)
                                    :  +- Filter (9)
                                    :     +- Scan csv  (8)
                                    +- ObjectHashAggregate (18)
                         

## Συμπεράσματα

Οι χρόνοι που πετύχαμε χρησιμοποιώντας τις 4 δοθείσες στρατηγικές για το πρώτο join είναι:

| Join Strategy | Time (seconds) |
|---|---|
| BROADCAST | 33.36 |
| MERGE | 32.95 |
| SHUFFLE HASH | 28.10 |
| SHUFFLE REPLICATE NL | 27.93 |

Η στρατηγική join που πετυχαίνει την καλύτερη επίδοση είναι η **SHUFFLE REPLICATE NL** (*Shuffle Replicate Nested Loop*) με χρόνο 27.93 δευτερόλεπτα, έχοντας μικρή διαφορά από τη **SHUFFLE HASH** (*Shuffle Hash Join*) με 28.10 δευτερόλεπτα.

Στη συνέχεια δοκιμάσαμε να χρησιμοποιήσουμε για το γεωχωρικό join (*ST_Within*) το **Broadcast Index Join** αντί του **Range Join**:

| 1st Join Strategy | 2nd Join Strategy | Time (seconds) |
|---|---|---|
| SHUFFLE HASH | RANGE JOIN | 28.10 |
| SHUFFLE HASH | BROADCAST INDEX JOIN | 28.80 |
| SHUFFLE REPLICATE NL | RANGE JOIN | 27.93 |
| SHUFFLE REPLICATE NL | BROADCAST INDEX JOIN | 30.92 |

Συνεπώς η καταλληλότερη στρατηγική για την περίπτωσή μας είναι η **SHUFFLE_REPLICATE_NL**, με τη χρήση του optimized **Range Join** του Sedona για το 2ο join.